In [1]:
!pip install --upgrade google-cloud-bigquery panel db-dtypes


[notice] A new release of pip available: 22.1.2 -> 22.2.1
[notice] To update, run: pip install --upgrade pip


In [120]:
from google.cloud import bigquery
import os
import panel as pn
from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets
import pandas as pd
import datetime as dt

pn.extension()

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/jamestjw/Downloads/PrivateKey.json"

client = bigquery.Client()

SENTIMENT_TABLE = 'comp-6231-356417.Twitter.sentiment_analysis'
TWEETS_TABLE = 'comp-6231-356417.Twitter.Tweets'
CYBERBULLYING_VALUES = ['neutral', 'racism', 'sexism']
SARCASM_VALUES = ['normal', 'sarcasm']
EMOTION_VALUES = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']
MAX_ROWS = 15

In [121]:
# So that everything can be seen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def show_sample_data(date_range=None, sarcasm=None, cyberbullying=None, emotion=None):
    wheres = []
    if date_range:
        start_date, end_date = date_range
        wheres.append(f'DATE(Tweets.created_at) BETWEEN "{str(start_date.date())}" and "{str(end_date.date())}"')
        
    if sarcasm:
        wheres.append(f'sarcasm = "{sarcasm}"')
        
    if cyberbullying:
        wheres.append(f'cyberbullying = "{cyberbullying}"')
        
    if emotion:
        wheres.append(f'emotion = "{emotion}"')

    where_clause = f"WHERE {' AND '.join(wheres)}" if len(wheres) > 0 else ""
        
    query = f"""
        SELECT
            content,
            cyberbullying,
            sarcasm,
            emotion,
            Tweets.created_at
        FROM 
            {SENTIMENT_TABLE}
        JOIN
            {TWEETS_TABLE}
        USING
            (tweet_id)
        {where_clause}            
        LIMIT {MAX_ROWS}
    """
    query_results= client.query(query)
    return pn.pane.DataFrame(query_results.to_dataframe(), index=False, width=1000)

In [122]:
date_range_slider = pn.widgets.DateRangeSlider(
    name='Date Range Slider',
    start=dt.datetime(2022, 5, 1), end=dt.datetime(2022, 8, 3),
    value=(dt.datetime(2022, 5, 1), dt.datetime(2022, 5, 2)),
)

sarcasm_choices = pn.widgets.RadioButtonGroup(name='Sarcasm', options=SARCASM_VALUES+[None], button_type='success')
cyberbullying_choices = pn.widgets.RadioButtonGroup(name='Cyberbullying', options=CYBERBULLYING_VALUES+[None], button_type='success')
emotion_choices = pn.widgets.RadioButtonGroup(name='Emotion', options=EMOTION_VALUES+[None], button_type='success')


interact(show_sample_data,
         date_range=date_range_slider,
         sarcasm=sarcasm_choices,
         emotion=emotion_choices,
         cyberbullying=cyberbullying_choices,
         manual_update=True)

Column
    [0] Column
        [0] DateRangeSlider(end=datetime.datetime(2022, ..., name='Date Range Slider', start=datetime.datetime(2022, ..., value=(datetime.datetime(2022, ..., value_end=datetime.datetime(2022, ..., value_start=datetime.datetime(2022, ...)
        [1] RadioButtonGroup(button_type='success', name='Sarcasm', options=['normal', 'sarcasm', ...])
        [2] RadioButtonGroup(button_type='success', name='Cyberbullying', options=['neutral', 'racism', ...])
        [3] RadioButtonGroup(button_type='success', name='Emotion', options=['sadness', 'joy', ...])
        [4] Button(name='Run Interact')
    [1] Row
        [0] DataFrame(DataFrame, index=False, width=1000)